

> Ver documentación en: [acerca de los transformers](https://medium.com/data-science/a-complete-guide-to-bert-with-code-9f87602e4a11)



Instalar dependencias:

In [ ]:
!pip -q install transformers datasets scikit-learn pandas

## Cargar el dataset

El fichero del dataset está almacenado dentro de la carpeta 'Rest-Mex_IberLEF' en Gooogle Drive.

In [ ]:
from google.colab import drive
# monta tu unidad de google drive
drive.mount('/content/drive')

# cambiamos el directorio de trabajo a la carpeta donde está almacenado el dataset
import os
os.chdir('/content/drive/My Drive/Rest-Mex_IberLEF/')

ModuleNotFoundError: No module named 'google'

Cargamos los conjuntos de datos de training y evaluación en dos objetos dataframes de pandas:

In [ ]:
import pandas as pd

df = pd.read_csv("Rest-Mex_2025_train.csv",  sep=',', on_bad_lines='skip')
print('tamaño del dataset:', df.shape)

#show the first rows
df.head()

tamaño del dataset: (208051, 6)


,Title,Review,Polarity,Town,Region,Type
0,Mi Lugar Favorito!!!!,Excelente lugar para comer y pasar una buena n...,5.0,Sayulita,Nayarit,Restaurant
1,lugares interesantes para visitar,"andar mucho, así que un poco difícil para pers...",4.0,Tulum,QuintanaRoo,Attractive
2,No es el mismo Dreams,"Es nuestra cuarta visita a Dreams Tulum, elegi...",3.0,Tulum,QuintanaRoo,Hotel
3,un buen panorama cerca de CancÃºn,"Estando en CancÃºn, fuimos al puerto y tomamos...",4.0,Isla_Mujeres,QuintanaRoo,Attractive
4,El mejor,Es un lugar antiguo y por eso me encanto tiene...,5.0,Patzcuaro,Michoacan,Hotel




> Anotaciones para tener en cuenta para la tarea de los pueblos mágicos:

*   tfidf - wordcount --> para las regiones
*   Elemento de lista


graficar si los word embedding se separan, en caso de hacer un cluster
tsne, psa, (reducción de la dimensionalidad), al menos para descaartar los embbedings, usasr el embbeding en español

Preparación del dataset:

In [ ]:
from sklearn.model_selection import train_test_split

# Drop rows with missing values
df.dropna(inplace=True)
# Merge title and review
df['text'] = df['Title'].fillna('') + ". " + df['Review'].fillna('')

# Encode labels
label_map = {'Hotel': 0, 'Restaurant': 1, 'Attractive': 2}
df['label'] = df['Type'].map(label_map)

df.head()

# Development dataset
df_big, df_dev = train_test_split(df, test_size=0.30, stratify=df['label'], random_state=42)

# Stratified Split (70/15/15)
train_df, temp_df = train_test_split(df_dev, test_size=0.30, stratify=df_dev['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

print(f"Train size: {len(train_df)} | Val size: {len(val_df)} | Test size: {len(test_df)}")


Train size: 43690 | Val size: 9362 | Test size: 9363


Tokenizando el dataset:

In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

def tokenize_fn(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

# Convert to HuggingFace Datasets
train_dataset = Dataset.from_pandas(train_df[['text', 'label']])
val_dataset = Dataset.from_pandas(val_df[['text', 'label']])
test_dataset = Dataset.from_pandas(test_df[['text', 'label']])

train_dataset = train_dataset.map(tokenize_fn, batched=True)
val_dataset = val_dataset.map(tokenize_fn, batched=True)
test_dataset = test_dataset.map(tokenize_fn, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

Map:   0%|          | 0/43690 [00:00<?, ? examples/s]

Map:   0%|          | 0/9362 [00:00<?, ? examples/s]

Map:   0%|          | 0/9363 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset)
print(val_dataset)
print(test_dataset)
#.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 43690
})
Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 9362
})
Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 9363
})


Cargando el modelo de Fine-Tuning:



> Token para loggearse en el wandb: 1fc8ae632543a17274762235fb29008a3a1d7ddd



In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

model = AutoModelForSequenceClassification.from_pretrained(
    "dccuchile/bert-base-spanish-wwm-cased", num_labels=3
)

training_args = TrainingArguments(
    report_to="none",
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

# metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds, average='weighted'),
    }


# trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.evaluate(test_dataset)

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-480f407e1e92>:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: wlxtbafvynqgjgsfyl (wlxtbafvynqgjgsfyl-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


In [ ]:
# simulate real test input (text only, no labels)
real_test = test_df.copy()
real_test = real_test.reset_index(drop=True)
real_test['InstanceID'] = [f"id{i:06d}" for i in range(len(real_test))]

test_text_dataset = Dataset.from_pandas(real_test[['text']])
test_text_dataset = test_text_dataset.map(tokenize_fn, batched=True)

predictions = trainer.predict(test_text_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)

# Reverse mapping
inv_label_map = {v: k for k, v in label_map.items()}
real_test['PredictedType'] = [inv_label_map[p] for p in predicted_labels]

# Format Output
task_name = "TripSentiment"
output_file = "avyus_runx.txt"

with open(output_file, 'w', encoding='utf-8') as f:
    for _, row in real_test.iterrows():
        f.write(f'"{task_name}"\t"{row["InstanceID"]}"\t"{row["PredictedType"]}"\n')

print(f"Output file written: {output_file}")